<a target="_blank" href="https://colab.research.google.com/github/ZHAW-ZAV/TSO-FS25-students/blob/main/02_python_viz/02_01_intro_plotly.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [ ]:
import sys
import os

IN_COLAB = "google.colab" in sys.modules

file_id_co2 = "1-FxcOXUXMrXZzDzeLgarKoQfa2QN_sXO"
file_id_trajs = "1CRNprh5oSxZntkmEJbCMTN3Htf5cPb4u"

if IN_COLAB:
    path_to_co2 = "/content/data/co2_emmissions_by_state.csv"
    os.makedirs(os.path.dirname(path_to_co2), exist_ok=True)
    !gdown "https://drive.google.com/uc?id={file_id_co2}" -O "{path_to_co2}"

    path_to_trajs = "/content/data/plotly_trajs.csv"
    os.makedirs(os.path.dirname(path_to_trajs), exist_ok=True)
    !gdown "https://drive.google.com/uc?id={file_id_trajs}" -O "{path_to_trajs}"

    # Unfortunately, Colab does not come with the Kaleido library installed.
    !pip install -q kaleido
else:
    import gdown

    url = f"https://drive.google.com/uc?id={file_id_co2}"
    path_to_co2 = "data/co2_emmissions_by_state.csv"
    os.makedirs(os.path.dirname(path_to_co2), exist_ok=True)
    gdown.download(url, path_to_co2, quiet=False)

    url = f"https://drive.google.com/uc?id={file_id_trajs}"
    path_to_trajs = "data/plotly_trajs.csv"
    os.makedirs(os.path.dirname(path_to_trajs), exist_ok=True)
    gdown.download(url, path_to_trajs, quiet=False)


The code above loads the data, don't modify.

---------------

***Notebook starts here***

In [39]:
from plotly import graph_objects as go
from plotly import express as px
import pandas as pd

## First steps with Plotly, scatter plots

In [ ]:
# We will use the iris dataset for this example
df = px.data.iris()
df

In [ ]:
fig = px.scatter(df, x="sepal_width", y="sepal_length", color="petal_length")
fig.show()

In [ ]:
df = px.data.iris()
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=df["sepal_width"],
        y=df["sepal_length"],
        marker_color=df["petal_length"],
        mode="markers",
        marker=dict(showscale=True),
    )
)
fig.show()


In [ ]:
# let's make this plot a bit more interesting
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=df["sepal_width"],
        y=df["sepal_length"],
        marker_color=df["petal_length"],
        mode="markers",
        marker=dict(showscale=True, colorbar=dict(title="Petal Length")),
    )
)
fig.update_layout(
    height=500,
    width=700,
    title="Iris dataset",
    xaxis_title="Sepal Width",
    yaxis_title="Sepal Length",
    xaxis=dict(showgrid=False),
    yaxis=dict(showgrid=False),
)
fig.show()

In [ ]:
# what happens if we have the color with a categorical variable?
fig = px.scatter(df, x="sepal_width", y="sepal_length", color="species")
fig.show()

In [ ]:
fig = go.Figure()
species = df["species"].unique()
for s in species:
    df_to_plot = df.query("species == @s")
    fig.add_trace(
        go.Scatter(
            x=df_to_plot["sepal_width"],
            y=df_to_plot["sepal_length"],
            mode="markers",
            name=s,
        )
    )

fig.update_layout(
    title="Iris dataset",
    xaxis_title="Sepal Width",
    yaxis_title="Sepal Length",
    legend=dict(title="Species"),
)
fig.show()

## Some more plots with the CO2 data

In [ ]:
df = pd.read_csv(path_to_co2)
df

In [47]:
# since we have too many states, let's filter the data to only keep 4 states
to_keep = ["SWITZERLAND", "GERMANY", "ITALY", "FRANCE"]
df = df.query("STATE_NAME in @to_keep")

In [48]:
# we want to plot the CO2 emissions by year and state
# for that, we need to sum the CO2 emissions by year and state first
df = df.groupby(["YEAR", "STATE_NAME"])["CO2_QTY_TONNES"].sum().reset_index()

In [ ]:
# let's do a bar plot
# the bars can be either grouped (barmode='group') or stacked (barmode='stack')
fig = px.bar(df, x="YEAR", y="CO2_QTY_TONNES", color="STATE_NAME", barmode="stack")
fig.show()

In [ ]:
fig = go.Figure()
states = df["STATE_NAME"].unique()
for s in states:
    df_to_plot = df.query("STATE_NAME == @s")
    fig.add_trace(go.Bar(x=df_to_plot["YEAR"], y=df_to_plot["CO2_QTY_TONNES"], name=s))
fig.update_layout(
    title="CO2 emissions by state",
    xaxis_title="Year",
    yaxis_title="CO2 emissions (tonnes)",
    legend=dict(title="State"),
    barmode="stack",
)
fig.show()

In [ ]:
# let's have a look at a flight trajectory
df = pd.read_csv(path_to_trajs).query("flight_id == 'EDW24_20180111'")
df

In [ ]:
fig = px.scatter_map(df, lat="latitude", lon="longitude", color="altitude")
fig.update_layout(
    height=600,
    width=800,
    map=dict(
        center=dict(lat=df["latitude"].mean(), lon=df["longitude"].mean()), zoom=4.5
    ),
)
fig.show()

In [ ]:
fig = go.Figure()
fig.add_trace(
    go.Scattermap(
        lat=df["latitude"],
        lon=df["longitude"],
        marker=dict(size=10, color=df["altitude"], showscale=True),
    )
)
fig.update_layout(
    height=600,
    width=800,
    map=dict(
        center=dict(lat=df["latitude"].mean(), lon=df["longitude"].mean()), zoom=4.5
    ),
)
fig.show()

## Subplots

In [ ]:
from plotly.subplots import make_subplots

fig = make_subplots(rows=2, cols=1)

fig.add_trace(
    go.Scatter(
        x=df["timestamp"], y=df["altitude"], mode="lines+markers", name="baro altitude"
    ),
    row=1,
    col=1,
)
fig.add_trace(
    go.Scatter(
        x=df["timestamp"], y=df["groundspeed"], mode="lines+markers", name="groundspeed"
    ),
    row=2,
    col=1,
)
fig.update_layout(height=600, width=800, title_text="Flight trajectory")
fig.show()

## Export

In [56]:
fig.write_image("my_image.png")
fig.write_image("my_image.pdf")
fig.write_html("my_image.html")